In [1]:
import numpy as np
import pandas as pd

import itertools

import torch
import torch.nn as nn
import torch.optim as optim

from hypll import nn as hnn
from hypll.tensors import TangentTensor
from hypll.optim import RiemannianAdam
from hypll.manifolds.poincare_ball import Curvature, PoincareBall

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.metrics import r2_score
from sklearn.model_selection import KFold

In [2]:
import sys
sys.path.append('../')
import util

In [93]:
df_train_X = pd.read_csv('../data/beer_features_train_samples_small.csv', index_col=0)
df_train_y = pd.read_csv('../data/beer_labels_panel_train_samples_small.csv', index_col=0)
df_val_X = pd.read_csv('../data/beer_features_train.csv', index_col=0)
df_val_y = pd.read_csv('../data/beer_labels_panel_train.csv', index_col=0)
df_test_X = pd.read_csv('../data/beer_features_test.csv', index_col=0)
df_test_y = pd.read_csv('../data/beer_labels_panel_test.csv', index_col=0)

train_X = df_train_X.values
train_y = df_train_y.values
val_X = df_val_X.values
val_y = df_val_y.values
test_X = df_test_X.values
test_y = df_test_y.values

train_X.shape, train_y.shape, val_X.shape, val_y.shape, test_X.shape, test_y.shape

((1750, 231), (1750, 50), (175, 231), (175, 50), (75, 231), (75, 50))

In [94]:
df_train_y

,A_malt_all,A_malt_grain,A_malt_bread,A_malt_cara,A_malt_burn,A_hops_all,A_hops_citrus,A_hops_tropical,A_hops_noble,A_hops_woody,...,coriander,clove,lactic,acetic,barnyard,alcohol,aftertaste,body,co2,overall
0,-0.941996,-1.075726,-0.285841,-0.178741,-0.604294,-0.656378,0.099024,0.241602,-1.268708,-0.956593,...,-0.348461,-0.286272,-0.443933,-0.369483,-0.281909,1.091877,0.162093,0.425791,0.425166,0.710623
1,-0.893153,-0.977069,-0.195571,-0.179653,-0.543785,-0.639002,0.101024,0.256538,-1.409950,-0.906288,...,-0.321335,-0.321205,-0.399579,-0.355341,-0.291434,1.023505,0.163216,0.361180,0.390275,0.744412
2,-0.880786,-1.137411,-0.336594,-0.230738,-0.601020,-0.634813,0.015769,0.156042,-1.255992,-0.990153,...,-0.310829,-0.351813,-0.351749,-0.403402,-0.345381,1.085524,-0.024607,0.490808,0.432825,0.715099
3,-0.879230,-1.071754,-0.219354,-0.193606,-0.624828,-0.643238,0.045454,0.138254,-1.286502,-0.981138,...,-0.393938,-0.283200,-0.350663,-0.354406,-0.254126,0.973770,-0.046796,0.374827,0.449950,0.686550
4,-0.875527,-1.063935,-0.205753,-0.175031,-0.584146,-0.621780,0.057810,0.077312,-1.368296,-0.878475,...,-0.348944,-0.363667,-0.395459,-0.270739,-0.279900,0.943100,-0.024269,0.404060,0.364624,0.793018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745,-1.233758,-0.868750,-0.638104,-0.875494,-0.278827,0.173468,1.801565,0.154312,-0.052745,-0.926968,...,1.335555,0.253309,-0.447637,-0.364690,-0.689424,-0.543003,-1.372243,-1.187967,0.816348,0.139063
1746,-1.063261,-0.751914,-0.690999,-0.924281,-0.199706,0.088800,1.762356,0.196654,-0.057584,-0.970705,...,1.251194,0.250426,-0.419021,-0.379580,-0.608952,-0.622449,-1.400643,-1.032351,0.763097,0.176882
1747,-1.153341,-0.802079,-0.600642,-0.885723,-0.224749,0.186860,1.835002,0.190688,-0.184398,-0.911978,...,1.219650,0.254567,-0.426003,-0.321299,-0.669511,-0.423319,-1.300411,-0.964460,0.593767,0.137067
1748,-1.203214,-0.841736,-0.702683,-0.924173,-0.245043,0.189425,1.795088,0.202355,-0.090596,-0.851933,...,1.240388,0.291119,-0.420381,-0.401480,-0.575356,-0.562966,-1.437035,-1.153961,0.783051,0.162953


In [95]:
FOLDS = 5
NUM_SAMPLE_TYPES = len(val_X)
NUM_SAMPLES_PER_TYPE = len(train_X) // NUM_SAMPLE_TYPES

fold_nums = list(range(FOLDS))
[num*NUM_SAMPLE_TYPES for num in fold_nums]
[(num+1)*NUM_SAMPLE_TYPES for num in fold_nums]

# FOLD_INDICES = util.get_fold_indices(NUM_SAMPLE_TYPES, FOLDS)

# def get_fold_indices(num_types, num_per_type, k, seed=42):
#     def get_val_start_ends(size, k):
#         fold_size = size // k
#         rest = size % k

#         fold_sizes = [fold_size] * k

#         for i in range(rest):
#             fold_sizes[i] += 1

#         indices = np.cumsum([fold_sizes])

#         return list(zip(indices-np.array(fold_sizes), indices))


#     np.random.seed(seed)
#     indices = np.random.random(num_types).argsort()

#     val_start_ends = get_val_start_ends(num_types, k)
#     val_indices = [indices[start:end] for start, end in val_start_ends]

#     train_indices = [list(set(range(num_types)) - set(val_is)) for val_is in val_indices]
#     exp_train_indices = [[list(range(val_i*num_per_type,(val_i+1)*num_per_type)) for val_i in val_is] for val_is in train_indices]

#     return np.array(val_indices), [np.array(exp_is).flatten() for exp_is in exp_train_indices]

val_indices, train_indices = util.get_fold_indices_rand(NUM_SAMPLE_TYPES, NUM_SAMPLES_PER_TYPE, FOLDS)
train_indices
# print(FOLD_INDICES)

[array([   0,    1,    2, ..., 1747, 1748, 1749]),
 array([   0,    1,    2, ..., 1747, 1748, 1749]),
 array([  10,   11,   12, ..., 1747, 1748, 1749]),
 array([   0,    1,    2, ..., 1737, 1738, 1739]),
 array([   0,    1,    2, ..., 1747, 1748, 1749])]

In [96]:
# Define custom PyTorch dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

<h1> Hyperbolic </h1>

In [97]:
# Define your MLP model
class HYP_MLP(nn.Module):
    def __init__(self, input_size, output_size, layer_size, num_hidden_layers, manifold):
        super(HYP_MLP, self).__init__()
        torch.manual_seed(42)
        self.fc_in = hnn.HLinear(input_size, layer_size, manifold=manifold)
        self.relu = hnn.HReLU(manifold=manifold)
        self.hidden_fcs = nn.ModuleList([hnn.HLinear(layer_size, layer_size, manifold=manifold) for _ in range(num_hidden_layers)])
        self.fc_out = hnn.HLinear(layer_size, output_size, manifold=manifold)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.relu(x)
        for fc in self.hidden_fcs:
            x = fc(x)
            x = self.relu(x)
        x = self.fc_out(x)

        return x


# Define training function
def hyp_train_model(model, train_loader, criterion, optimizer, manifold, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        tangents = TangentTensor(data=inputs, man_dim=-1, manifold=manifold)
        manifold_inputs = manifold.expmap(tangents)

        outputs = model(manifold_inputs)

        loss = criterion(outputs.tensor, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

<h1> EUCLIDEAN </h1>

In [98]:
# Define your MLP model
class EUC_MLP(nn.Module):
    def __init__(self, input_size, output_size, layer_size, num_hidden_layers):
        super(EUC_MLP, self).__init__()
        torch.manual_seed(42)
        self.fc_in = nn.Linear(input_size, layer_size)
        self.relu = nn.ReLU()
        self.hidden_fcs = nn.ModuleList([nn.Linear(layer_size, layer_size) for _ in range(num_hidden_layers)])
        self.fc_out = nn.Linear(layer_size, output_size)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.relu(x)
        for fc in self.hidden_fcs:
            x = fc(x)
            x = self.relu(x)
        x = self.fc_out(x)

        return x

# Define training function
def euc_train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

In [99]:
param_grid = {
    'model_type': ['hyp', 'euc'],
    'num_hidden_layers': [0,1,2,4,8],
    'layer_size': [32,64,128,256],
    'lr': [0.02],
    'weight_decay': [0.005],
    'batch_size': [1024],
    'epochs': [100],
    'curvature': [-1]
}

param_combinations = list(itertools.product(*param_grid.values()))
len(param_combinations)

40

In [100]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

param_eval_stats = []

for i, params in enumerate(param_combinations):
    print(f'----- Combination {i} -----')
    print(*zip(param_grid.keys(), params))
    model_type, num_hidden_layers, layer_size, lr, weight_decay, batch_size, epochs, curvature = params

    # for fold, (fold_train_indices, fold_val_indices) in enumerate(zip(train_indices, val_indices)):
    #     print(f'Fold {fold}')

    #     fold_train_X = train_X[fold_train_indices]
    #     fold_train_y = train_y[fold_train_indices]
    #     fold_val_X   = val_X[fold_val_indices]
    #     fold_val_y   = val_y[fold_val_indices]
    #     # fold_val_X   = test_X
    #     # fold_val_y   = test_y
        
    for fold in [0]:
        print(f'Fold {fold}')

        fold_train_X = train_X
        fold_train_y = train_y
        fold_val_X   = test_X
        fold_val_y   = test_y

        train_dataset = CustomDataset(fold_train_X, fold_train_y)
        val_dataset = CustomDataset(fold_val_X, fold_val_y)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        if model_type == 'hyp':
            manifold = PoincareBall(c=Curvature(curvature))
        elif model_type == 'euc':
            manifold = None

        if model_type == 'hyp':
            model = HYP_MLP(input_size=train_X.shape[1],
                            output_size=train_y.shape[1],
                            layer_size=layer_size,
                            num_hidden_layers=num_hidden_layers,
                            manifold=manifold).to(device)
        elif model_type == 'euc':
            model = EUC_MLP(input_size=train_X.shape[1],
                            output_size=train_y.shape[1],
                            layer_size=layer_size,
                            num_hidden_layers=num_hidden_layers).to(device)

        criterion = nn.MSELoss()

        if model_type == 'hyp':
            optimizer = RiemannianAdam(model.parameters(), lr=lr, weight_decay=weight_decay)
        elif model_type == 'euc':
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        eval_stats = {'loss': {'train': [], 'val': []}, 'mae': {'train': [], 'val': []}}

        for epoch in range(epochs):
            if model_type == 'hyp':
                eval_stats['loss']['train'].append(hyp_train_model(model, train_loader, criterion, optimizer, manifold, device))
                eval_stats['loss']['val'].append(util.h_evaluate_loss(model, val_loader, criterion, manifold, device))

                eval_stats['mae']['train'].append(util.h_evaluate_r2(model, train_loader, manifold, device))
                eval_stats['mae']['val'].append(util.h_evaluate_r2(model, val_loader, manifold, device))
            elif model_type == 'euc':
                eval_stats['loss']['train'].append(euc_train_model(model, train_loader, criterion, optimizer, device))
                eval_stats['loss']['val'].append(util.evaluate_loss(model, val_loader, criterion, device))

                eval_stats['mae']['train'].append(util.evaluate_r2(model, train_loader, device))
                eval_stats['mae']['val'].append(util.evaluate_r2(model, val_loader, device))

        print(eval_stats['mae']['val'])
        param_eval_stats.append(eval_stats)

----- Combination 0 -----
('model_type', 'hyp') ('num_hidden_layers', 0) ('layer_size', 32) ('lr', 0.02) ('weight_decay', 0.005) ('batch_size', 1024) ('epochs', 100) ('curvature', -1)
Fold 0


KeyboardInterrupt: 

<h1>BEST MODEL</h1>

In [24]:
best_params = {
    'model_type': 'hyp',
    'num_hidden_layers': 4,
    'layer_size': 256,
    'lr': 0.003,
    'weight_decay': 0.001,
    'batch_size': 1024,
    'epochs': 10,
    'curvature': -1
}

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_type, num_hidden_layers, layer_size, lr, weight_decay, batch_size, epochs, curvature = best_params.values()



kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(train_X, train_y)):
    print(f'Fold {fold}')
    # fold_train_X, fold_val_X = train_X[train_idx], train_X[val_idx]
    # fold_train_y, fold_val_y = train_y[train_idx], train_y[val_idx]
    fold_train_X, fold_val_X = train_X[train_idx], test_X
    fold_train_y, fold_val_y = train_y[train_idx], test_y
    # fold_train_X, fold_val_X = train_X, test_X
    # fold_train_y, fold_val_y = train_y, test_y

    train_dataset = CustomDataset(fold_train_X, fold_train_y)
    val_dataset = CustomDataset(fold_val_X, fold_val_y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    if model_type == 'hyp':
        manifold = PoincareBall(c=Curvature(curvature))
    elif model_type == 'euc':
        manifold = None

    if model_type == 'hyp':
        model = HYP_MLP(input_size=train_X.shape[1],
                        output_size=train_y.shape[1],
                        layer_size=layer_size,
                        num_hidden_layers=num_hidden_layers,
                        manifold=manifold).to(device)
    elif model_type == 'euc':
        model = EUC_MLP(input_size=train_X.shape[1],
                        output_size=train_y.shape[1],
                        layer_size=layer_size,
                        num_hidden_layers=num_hidden_layers).to(device)

    criterion = nn.MSELoss()

    if model_type == 'hyp':
        optimizer = RiemannianAdam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif model_type == 'euc':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    eval_stats = {'loss': {'train': [], 'val': []}, 'mae': {'train': [], 'val': []}}

    for epoch in range(epochs):
        if model_type == 'hyp':
            eval_stats['loss']['train'].append(hyp_train_model(model, train_loader, criterion, optimizer, manifold, device))
            eval_stats['loss']['val'].append(util.h_evaluate_loss(model, val_loader, criterion, manifold, device))

            eval_stats['mae']['train'].append(util.h_evaluate_r2(model, train_loader, manifold, device))
            eval_stats['mae']['val'].append(util.h_evaluate_r2(model, val_loader, manifold, device))
        elif model_type == 'euc':
            eval_stats['loss']['train'].append(euc_train_model(model, train_loader, criterion, optimizer, device))
            eval_stats['loss']['val'].append(util.evaluate_loss(model, val_loader, criterion, device))

            eval_stats['mae']['train'].append(util.evaluate_r2(model, train_loader, device))
            eval_stats['mae']['val'].append(util.evaluate_r2(model, val_loader, device))

    print(eval_stats['mae']['val'])

Fold 0
[0.10765948756282011, 0.12308321606784109, 0.12454766437259072, 0.12698461687460463, 0.12526742669738572, 0.12486854942081814, 0.12370138416228064, 0.12335186350024209, 0.12311975745553928, 0.12481188844227081]
Fold 1
[0.10694572659119995, 0.12300988018483655, 0.12596251597252395, 0.12713697133334384, 0.12319071662434075, 0.1237505160267185, 0.12400483580433477, 0.12384196969943467, 0.1234383582915921, 0.12249493166258585]
Fold 2
[0.10687856100840314, 0.12332382928659225, 0.12391855430801797, 0.12579272453833407, 0.12444610385627473, 0.12367803361043006, 0.12372331089353018, 0.1233711942192049, 0.1228020182704326, 0.12271869834607192]
Fold 3
[0.10750576340391302, 0.12294992224508638, 0.12448672795407086, 0.12575785160685407, 0.12344712346292308, 0.12408783516163593, 0.12411291205089521, 0.12396073754233594, 0.12467454068302844, 0.1239832776480632]
Fold 4
[0.106911679887455, 0.12443725528270613, 0.1259176711359262, 0.12665070801147957, 0.12356156026727742, 0.12352775207013689, 0.

In [12]:
1000
[0.09094981766612582, 0.09608602443769439, 0.10213084733073284, 0.10297098476158782, 0.1039252772204806, 0.10574258559121522, 0.10593447081943014, 0.1103528270397624, 0.10905810868205902, 0.11674655082317638]
[0.09124620536624219, 0.09588467278224783, 0.1109519321222205, 0.10871296762480671, 0.11322182908185724, 0.10604926039894355, 0.11641335699831064, 0.11669891168893905, 0.1163291967061507, 0.11563811270315874]
[0.08261279344412803, 0.0949209955128152, 0.10394174902203669, 0.11074634671414854, 0.10959598998285282, 0.10492545479477039, 0.10590842284461503, 0.10538532105083259, 0.10268829907521582, 0.10696282556494001]
[0.09143749197812866, 0.10686854162056632, 0.10184344301563328, 0.10983260881492879, 0.11453995365690633, 0.10643641196149398, 0.10807662045371369, 0.11264002132608508, 0.11569218070241385, 0.11172063540914148]
[0.08755047684265042, 0.09826624201788955, 0.10408250848514021, 0.10678387971099373, 0.10168236861206509, 0.09372892354517087, 0.10054121141766975, 0.11307510662137904, 0.10818904212215745, 0.11473416400049585]

100
[0.10712686537621345, 0.12349026400269329, 0.1246606174341069, 0.12693350344457163, 0.12540473804914373, 0.12392351307841984, 0.12310184680085018, 0.1235122503852268, 0.12269077053490243, 0.12184334164640719]
[0.10825157818713378, 0.12289771182725055, 0.12439158651151372, 0.1261395813343429, 0.1253862140092597, 0.12366027630753869, 0.12406626861964988, 0.1242455269123071, 0.12363573750877084, 0.12229140361532519]
[0.10685337929854738, 0.12412607759674942, 0.12566753200057623, 0.12656694272727384, 0.1249030203813584, 0.12408679288839279, 0.12404666634192532, 0.12445838543532435, 0.12457812238461687, 0.12304167921955317]
[0.10745761389567116, 0.12397573098456129, 0.12521314400651082, 0.1276813410290952, 0.1252595522840151, 0.12429336231380055, 0.12474528515336958, 0.1244248228708727, 0.12511605799175451, 0.12342070994183856]
[0.10759642672103734, 0.12380378206783893, 0.12494015763741242, 0.12673535494694677, 0.12506894028324306, 0.12376698712522265, 0.12254735888244327, 0.12283803721108438, 0.12404691001407472, 0.12351575220797617]

50
[0.07677710553661399, 0.10584953169202371, 0.11966844890984371, 0.12411849676234876, 0.12520028446868986, 0.12418479259928074, 0.12732371128638775, 0.126271569310595, 0.12580454850610243, 0.12429738209045665]
[0.07721830195010819, 0.10745842540642501, 0.12112070383855837, 0.12386530838495947, 0.125923156127822, 0.1258847214082912, 0.12830232653117135, 0.12669207756182055, 0.12631119756542175, 0.12450957025993643]
[0.07632709753632146, 0.10763185461397703, 0.1186022848413865, 0.12323234538214417, 0.12484191533525628, 0.12444830403759084, 0.12718327433134718, 0.12617910882856184, 0.12485011626791608, 0.1229076001872507]
[0.0767599902123989, 0.1089768569863075, 0.120941318864213, 0.12583457699099382, 0.1255480130415434, 0.12394682506457874, 0.12700223229913882, 0.12678108092511747, 0.12549870300322666, 0.12525940753014855]
[0.07607748019287443, 0.10721069870104394, 0.12062286165389509, 0.12380926861086566, 0.1258001163362215, 0.12521044913472032, 0.1273641772466992, 0.1263506895116415, 0.12492659096620544, 0.12464252907020315]

50